#  <div style="text-align:center"><u>Realtime Face Swapping</u></div>
<table style="width:100%; border: solid 1px black">
  <tr style="border: solid 1px black">
    <td style="text-align:left; border: solid 1px black"><b>Michael Adriel Darmawan</b></td>
    <td style="text-align:left; border: solid 1px black"><b>Juan Farell Haryanto</b></td>
    <td style="text-align:left; border: solid 1px black"><b>Jordan Jonathan Gouw</b></td>
  </tr>
  <tr style="border: solid 1px black">
    <td style="text-align:left; border: solid 1px black"><b>2301854170</b></td>
    <td style="text-align:left; border: solid 1px black"><b>2301855072</b></td>
    <td style="text-align:left; border: solid 1px black"><b>2301852291</b></td>
  </tr>
   <tr style="border: solid 1px black">
    <td style="text-align:left; border: solid 1px black"><b>LA02</b></td>
    <td style="text-align:left; border: solid 1px black"><b>LA02</b></td>
    <td style="text-align:left; border: solid 1px black"><b>LA02</b></td>
  </tr>
</table>

1. Importing Libraries

In [16]:
import cv2
import numpy as np
import dlib

2. Define a function to extract an index from numpy array

In [17]:
def extractIndexFromNumpyArray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index

3. Prepare source image and convert it to grayscale
4. Create a mask of the same size as the source image
5. Create face detector instance and facial landmark points predictor instance 

In [18]:
sourceImg = cv2.imread("faces/shaq.jpg")
sourceImgGray = cv2.cvtColor(sourceImg, cv2.COLOR_BGR2GRAY)
sourceImgMask = np.zeros_like(sourceImgGray)

vidCapture = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

6. Create an array to save the indexes of delaunay triangles points of the source image
7. Detect faces in the source image
8. For each detected face, extract the facial landmarks and append the points to the array, then convert the array to numpy array
9. Create a convex hull of the points of the face in the source image
10. Create a rectangle that bounds the convex hull
11. Using Subdiv2D, create a Delaunay triangulation of the convex hull inside the rectangle
12. Save triangle indexes of the Delaunay triangulation to the array, convert the array to numpy array
13. For each point of the triangle in the Delaunay triangulation that exist in the facial landmark points, extract the points of the triangle and append the points to the array (Finding the corrdinates of the facial landmarks)


In [19]:
delaunayTrianglesIndexes = []
faces = detector(sourceImgGray)
for face in faces:
    landmarks = predictor(sourceImgGray, face)
    landmarkPointsSource = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarkPointsSource.append((x, y))

    points = np.array(landmarkPointsSource, np.int32)
    convexhull = cv2.convexHull(points)
    cv2.fillConvexPoly(sourceImgMask, convexhull, 255)

    rect = cv2.boundingRect(convexhull)
    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(landmarkPointsSource)
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32)

    for t in triangles:
        point1 = (t[0], t[1])
        point2 = (t[2], t[3])
        point3 = (t[4], t[5])

        point1Index = np.where((points == point1).all(axis=1))
        point1Index = extractIndexFromNumpyArray(point1Index)

        point2Index = np.where((points == point2).all(axis=1))
        point2Index = extractIndexFromNumpyArray(point2Index)

        point3Index = np.where((points == point3).all(axis=1))
        point3Index = extractIndexFromNumpyArray(point3Index)

        if point1Index is not None and point2Index is not None and point3Index is not None:
            triangle = [point1Index, point2Index, point3Index]
            delaunayTrianglesIndexes.append(triangle)

14. Convert video capture to grayscale
15. Create a mask of the same size as the video capture
16. Detect faces in the video capture
17. For each detected face, extract the facial landmarks and append the points to the array, then convert the array to numpy array
18. Create a convex hull of the points of the face in the video capture
19. Loop through all triangles and draw the triangles based on each of the corrdinates for both the source image and the video capture  
20. Extract the points of the triangle and mask the triangles in the source image and the video capture
21. Using affine transformation, we warp the triangles in source image to match the triangles in the video capture
22. For each iteration of the triangles, we add the warped triangles of the source image to the video capture
23. Add median blur to the face
24. Put the face to the video capture by creating a mask first and then using the mask to put the face to the video capture
25. Use seamless clone to match the color of the face in the source image to the face in the video capture
26. Sharpening the image

In [20]:
while True:
    _, targetFace = vidCapture.read()
    targetFaceGray = cv2.cvtColor(targetFace, cv2.COLOR_BGR2GRAY)
    targetFaceMask = np.zeros_like(targetFace)

    faces2 = detector(targetFaceGray)
    for face in faces2:
        landmarkPointsTarget = []
        landmarks = predictor(targetFaceGray, face)
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmarkPointsTarget.append((x, y))

        pointsTarget = np.array(landmarkPointsTarget, np.int32)
        convexhull2 = cv2.convexHull(pointsTarget)

    # Triangulation of both faces
    for triangleIndex in delaunayTrianglesIndexes:
        # Triangulation of the first face
        sourcePoint1 = landmarkPointsSource[triangleIndex[0]]
        sourcePoint2 = landmarkPointsSource[triangleIndex[1]]
        sourcePoint3 = landmarkPointsSource[triangleIndex[2]]
        triangleSource = np.array([sourcePoint1, sourcePoint2, sourcePoint3], np.int32)

        rectangleSource = cv2.boundingRect(triangleSource)
        (x, y, w, h) = rectangleSource
        croppedTriangleSource = sourceImg[y: y + h, x: x + w]
        croppedTriangleSourceMask = np.zeros((h, w), np.uint8)

        pointsSource = np.array([[sourcePoint1[0] - x, sourcePoint1[1] - y],
                           [sourcePoint2[0] - x, sourcePoint2[1] - y],
                           [sourcePoint3[0] - x, sourcePoint3[1] - y]], np.int32)

        cv2.fillConvexPoly(croppedTriangleSourceMask, pointsSource, 255)

        # Triangulation of second face
        targetPoint1 = landmarkPointsTarget[triangleIndex[0]]
        targetPoint2 = landmarkPointsTarget[triangleIndex[1]]
        targetPoint3 = landmarkPointsTarget[triangleIndex[2]]
        triangleTarget = np.array([targetPoint1, targetPoint2, targetPoint3], np.int32)

        rectangleTarget = cv2.boundingRect(triangleTarget)
        (x, y, w, h) = rectangleTarget
        croppedTriangleTargetMask = np.zeros((h, w), np.uint8)

        pointsTarget = np.array([[targetPoint1[0] - x, targetPoint1[1] - y],
                            [targetPoint2[0] - x, targetPoint2[1] - y],
                            [targetPoint3[0] - x, targetPoint3[1] - y]], np.int32)

        cv2.fillConvexPoly(croppedTriangleTargetMask, pointsTarget, 255)

        # Warp triangles
        pointsSource = np.float32(pointsSource)
        pointsTarget = np.float32(pointsTarget)
        M = cv2.getAffineTransform(pointsSource, pointsTarget)
        warpedTriangle = cv2.warpAffine(croppedTriangleSource, M, (w, h))
        warpedTriangle = cv2.bitwise_and(
            warpedTriangle, warpedTriangle, mask=croppedTriangleTargetMask)

        # Reconstructing destination face
        targetFaceRectangle =targetFaceMask[y: y + h, x: x + w]
        targetFaceRectangleGray = cv2.cvtColor(
            targetFaceRectangle, cv2.COLOR_BGR2GRAY)
        _, maskedTriangle = cv2.threshold(
            targetFaceRectangleGray, 20, 255, cv2.THRESH_BINARY_INV)
        warpedTriangle = cv2.bitwise_and(
            warpedTriangle, warpedTriangle, mask=maskedTriangle)

        targetFaceRectangle = cv2.add(
            targetFaceRectangle, warpedTriangle)
        targetFaceRectangle = cv2.medianBlur(targetFaceRectangle, 3)
        targetFaceMask[y: y + h, x: x + w] = targetFaceRectangle

    # Face swapped (putting 1st face into 2nd face)
    resultFaceMask = np.zeros_like(targetFaceGray)
    resultHeadMask = cv2.fillConvexPoly(resultFaceMask, convexhull2, 255)
    resultFaceMask = cv2.bitwise_not(resultHeadMask)

    resultHeadOnly = cv2.bitwise_and(targetFace, targetFace, mask=resultFaceMask)
    result = cv2.add(resultHeadOnly, targetFaceMask)

    (x, y, w, h) = cv2.boundingRect(convexhull2)
    faceCenter = (int((x + x + w) / 2), int((y + y + h) / 2))
    seamlessClone = cv2.seamlessClone(
        result, targetFace, resultHeadMask, faceCenter, cv2.MIXED_CLONE)

    # create kernel to sharpen image with small effect
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    seamlessClone = cv2.filter2D(seamlessClone, -1, kernel)

    cv2.namedWindow('Face Swapped', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Face Swapped', 1600, 950)
    cv2.imshow('Face Swapped', seamlessClone)

    key = cv2.waitKey(1)
    if key == 27:
        break

vidCapture.release()
cv2.destroyAllWindows()